# Predicción de Precio de Casas

![](https://storage.googleapis.com/kaggle-datasets-images/128/270/d149695d1f9a97ec54cf673be6430ad7/dataset-original.jpg)

*Objetivo: predecir precio de casas en King County en el estado de Washington (data de [Kaggle](https://www.kaggle.com/harlfoxem/housesalesprediction)).*


Pensemos primero en un modelo lineal simple utilizando solamente los metros cuadrados:
$$ \log(price) = \beta_{0} + \beta_{1}m2 + \varepsilon $$

Interpretabilidad: la ordenada al origen es el precio inicial de una casa y la pendiente cuanto crece este por metro cuadrado.

En la misma línea: **¿por qué en logaritmos?** -> *Power Law/Fat Tails*

Para medir la bondad de ajuste usamos el error cuadrático medio (ECM):
$$ ECM = \frac{1}{N} \sum_{i=1}^{N}(y_{i} - \hat{f}(x_{i}))^{2} =
          \sum_{i=1}^{N}(\log(price_{i}) - \beta_{0} + \beta_{1}m2_{i})^{2} $$



## Modelo Lineal Univariado

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn import linear_model

### SIN DATOS NO HAY PARAISO
![](https://raw.githubusercontent.com/petobens/introduccion-ml-aplicado/master/figures/ml-claro-1/whatido.jpg)


#### Obteniendo el dataset de Kaggle

In [ ]:
from google.colab import files
files.upload()

In [ ]:
!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!ls ~/.kaggle
!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d harlfoxem/housesalesprediction

#### Cargar el dataset con Pandas

In [ ]:
def extract_housing_prices_data():
    #url_csv = 'https://storage.googleapis.com/qeds/data/kc_house_data.csv'
    df = pd.read_csv('housesalesprediction.zip')
    return df

In [ ]:
df = extract_housing_prices_data()

In [ ]:
df

In [ ]:
df.sample(10)

In [ ]:
df.info()

## Un poco de EDA

### EDA y ETFL

EDA: Exploratory Data Analysis/Análisis Estadístico Descriptivo

**Data Mining**: en su acepción pura son técnicas para analizar los datos, encontrar patrones y generar insights.

- **Visualizaciones:** univariadas (para un mismo feature), bi-variadas (entre features y target), multivariadas (entre distintos features)
- **Técnicas de estadística clásica:** correlaciones, test de hipótesis, ANOVA.
- **Reducción de dimensionalidad:** PCA, SVD
- **Clustering**

**Objetivos:**
- Comprender el dataset.
- Definir y refinar la selección e ingeniería de atributos que alimentan los modelos.


¿Qué involucra hacer data science en la industria?
**E**xtract **T**ransform **F**it **L**oad

Distribución de tiempos: **E 30\% | T 50\% | F 10\% | L 10\%**



In [ ]:
df['bedrooms'].value_counts().to_frame().reset_index().sort_values(by='index')

In [ ]:
df['price'].value_counts()

In [ ]:
df1 = df[['bedrooms', 'bathrooms', 'price']]
df1.hist(bins=25,xlabelsize='10',ylabelsize='10')

In [ ]:
df_corr = df.corr('pearson')
df_corr.style.background_gradient(cmap='coolwarm', axis=None)

In [ ]:
def scatter_wrapper(df, ax=None, x='sqft_living', y='price', color='b'):
    if ax is None:
        _, ax = plt.subplots(figsize=(8, 6))
    df.plot.scatter(x=x , y=y, c=color, alpha=0.35, s=1.5, ax=ax)
    return ax

In [ ]:
scatter_wrapper(df)

### Preprocesamiento

### Transformacion: Ingenieria

![](https://www.oreilly.com/library/view/feature-engineering-for/9781491953235/assets/feml_0102.png)

El "aprendizaje" del modelo dependerá de la información y variabilidad provista por los atributos (*Garbage in - garbage out*).

Existe gran variedad de técnicas para generar/modificar atributos

- **Transformaciones logaritmicas:** tiene sentido cuando variables siguen una distribución asimétrica positiva (masa concentrada en valores pequeños y grandes con poca densidad)
- **Reescalamiento de variables numéricas:** si el modelo es sensible a la escala del atributo es deseable hacer transformaciones tipo *estandarización*.
(Esto puede ser propenso a *data leakage*)
- **Binning:** agrupación en *bins* ordenados
- **One-hot encoding:** transformación de variables categóricas en variables contínuas. ![](https://raw.githubusercontent.com/petobens/introduccion-ml-aplicado/master/figures/ml-claro-1/one_hot.png)





In [ ]:
df['date']

In [ ]:
pd.to_datetime(df['date']).dt.month

In [ ]:
def preprocess_data(df):
    df['sales_yr'] = pd.to_datetime(df['date']).dt.year
    df['log_price'] = np.log(df['price'])
    return df

In [ ]:
df = preprocess_data(df)

In [ ]:
df.columns

In [ ]:
scatter_wrapper(df, y='log_price')

In [ ]:
def xy_split(df):
    y = df['log_price']
    X = df.drop(['price', 'log_price', 'date', 'id'], axis=1).copy()
    return X, y

In [ ]:
X, y = xy_split(df)

In [ ]:
y

In [ ]:
X.columns

In [ ]:
def fit(X, y, x_cols=None):
    lr_model = linear_model.LinearRegression()
    X = X if x_cols is None else X[x_cols]
    lr_model.fit(X, y)
    return lr_model

In [ ]:
lr_model = fit(X, y, x_cols=['sqft_living'])

In [ ]:
beta_0, beta_1 = lr_model.intercept_, lr_model.coef_[0]

In [ ]:
beta_0, beta_1

In [ ]:
# Plotear predicción
ax = scatter_wrapper(df, y='log_price')
x = np.array([0, df['sqft_living'].max()])
ax.plot(x, beta_0 + beta_1*x)

In [ ]:
def predict(model, X, exp=False):
    y_pred = model.predict(X)
    if exp:
        y_pred = np.exp(y_pred)
    return y_pred

In [ ]:
predict(lr_model, [[5000]], exp=True) # Nota: esto es un array

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
def score(y, y_hat):
    mse = mean_squared_error(y, y_hat)
    print(f"MSE is {mse}")
    return mse

In [ ]:
y_hat = predict(lr_model, X[['sqft_living']])

In [ ]:
y_hat

In [ ]:
score(y, y_hat)

## Ejercicios

### Ejercicios Regresion Lineal Simple

1. **Caching**
Modifique la función que extrae los datos de forma tal que guarde una copia local de los  mismos en caso que esta no exista aun. Si este archivo existe entonces devolver los datos ya guardados a menos que expresamente se quiere rehacer la consulta.
2. **Sistema métrico**
  1. Escriba una función que convierta metros cuadrados a pies cuadrados.
  2. Use la función anterior para estimar el precio de un vivienda con 464 m2 de living.
3. **Regresión lineal multivariada**
  1. Ajuste ahora un modelo con la totalidad de los features: $Y = X\beta + \varepsilon$
  2. En un mismo plano (`sqft_living`, `log_price`), grafique una nube de puntos con la data actual, los valores predicho por el modelo simple y aquellos predichos por este modelo multivariado.
  3. Compare el error cuadrático medio de los dos modelos.
  4. *Feature Engineering:* cree una nueva variable que refleje la fracción de pies no subterraneos y reestime el modelo con este nuevo atributo. ¿Encuentra mejoras en performance?
4. In Sample versus Out Of Sample
  1. Para pensar: ¿con lo hecho hasta ahora puede usted asegurar que su modelo performará correctamente con datos desconocidos?


### Ejercicio 1: Caching

In [ ]:
from pathlib import Path

def extract_housing_prices_data_with_cache(cache_fn='hp_data.csv', refresh_cache=False):
    ## TODO: implementar
    return df

In [ ]:
df_cache = extract_housing_prices_data_with_cache(refresh_cache=False)

### Ejercicio 2: Sistema metrico

In [ ]:
##TODO: implementar

In [ ]:
predict(lr_model, [[m2_to_sqft(464)]], exp=True)

### Ejercicio 3: Regresión multivariada y feature engineering

In [ ]:
lr_model_full = fit(X, y)

In [ ]:
x = 'sqft_living'
size = 1.5
ax = scatter_wrapper(df, y='log_price')
ax.scatter(X[x], predict(lr_model_full, X), c='r', s=size)
ax.scatter(X[x], predict(lr_model, X[[x]]), c='y', s=size)
ax.legend(['data', 'full', 'sqft'])

In [ ]:
X_fe = X.copy()
X_fe['pct_sqft_above'] = X_fe['sqft_above'] / X['sqft_living']

In [ ]:
X_fe.columns

In [ ]:
lr_model_full_above = fit(X_fe, y)

In [ ]:
y_hat_full_above = predict(lr_model_full_above, X_fe)
mse_full_above = score(y, y_hat_full_above)

In [ ]:
y_hat_full = predict(lr_model_full, X)
mse_full = score(y, y_hat_full)

## Seleccion de Modelos

### Selección de modelos: Conjunto de Validación

Hasta ahora hemos visto un único modelo y calculado una métrica de performance sobre el conjunto de entrenamiento
Pero... **queremos predecir bien sobre datos desconocidos!**


¿Cómo hacerlo? Simulamos la división entre datos conocidos y desconocidos.

*Enfoque de validation/holdout set:* entrenamos el modelo con datos conocidos y validamos las predicciones con el *conjunto de validación* (desconocido)

El conjunto de validación es una submuestra al azar de observaciones del conjunto de entrenamiento (usualmente 20%)

Posible problemas: 

1. La predicción de performance tiene alta variabilidad ya que depende de la participación de observaciones en entrenamiento/validación. 
2. Al achicar el conjunto de entrenamiento podemos estar sobrestimando el error de test.

![](https://raw.githubusercontent.com/petobens/introduccion-ml-aplicado/master/figures/ml-claro-1/holdout.png)

**¿Definido el modelo final: qué observaciones usamos para entrenarlo?**


In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

In [ ]:
def fit(X, y, x_cols=None, model='lr', max_depth=None):
    if model == 'lr':
        clf = linear_model.LinearRegression()
    elif model == 'dt':
        clf = DecisionTreeRegressor(max_depth=max_depth)
    elif model == 'rf':
        clf = RandomForestRegressor()
    else:
        raise ValueError("Unknown model!")

    X = X if x_cols is None else X[x_cols]
    clf.fit(X, y)
    return clf

In [ ]:
dt_model = fit(X, y, model='dt')

In [ ]:
dt_model

In [ ]:
# Train test split
test_n = 50
X_train = X.iloc[:test_n, :]
y_train = y.iloc[:test_n]
X_test = X.iloc[test_n:, :]
y_test = y.iloc[test_n:]

In [ ]:
X.shape, X_train.shape, X_test.shape

In [ ]:
def fit_pipeline(X_train, y_train, X_test, y_test, model, max_depth=None):
    m = fit(X_train, y_train, model=model, max_depth=max_depth)
    mses = {}
    for phase in ['train', 'test']:
        X = X_train if phase == 'train' else X_test
        y = y_train if phase == 'train' else y_test
        y_hat = predict(m, X)
        print(f"{phase} phase:")
        mses[phase] = score(y, y_hat)
    return mses

In [ ]:
fit_pipeline(X_train, y_train, X_test, y_test, 'lr')

In [ ]:
fit_pipeline(X_train, y_train, X_test, y_test, 'dt')

In [ ]:
# Hagamos tuning del hiperparámetro de profundidad de los árboles

In [ ]:
depths = range(1, 15)

In [ ]:
max_depths = range(1, 15)
df_mse = pd.DataFrame()
for max_depth in max_depths:
    mse = fit_pipeline(X_train, y_train, X_test, y_test, 'dt', max_depth=max_depth)
    mse['max_depth'] = max_depth
    df_mse = df_mse.append(mse, ignore_index=True)

In [ ]:
fig, ax = plt.subplots(figsize=(10,6))
df_mse.plot(x='max_depth', y='test', c='r', ax=ax)
df_mse.plot(x='max_depth', y='train', c='b', ax=ax)
ax.set_xlabel('max_depth')
ax.set_ylabel('ECM')

### Validación Cruzada

![]()![image.png](https://raw.githubusercontent.com/petobens/introduccion-ml-aplicado/master/figures/ml-claro-1/cv_k_fold.png)

¿Cómo podemos resolver los problemas de la estrategia de holdout?

Una opción es hacer **k-fold cross validation:**

1. Particionar el dataset en $k$ subconjuntos (folds) de tamaño parecido.
2. Usar el enésimo fold como conjunto de validación y el resto como conjunto de entrenamiento computando el ECM en cada caso.
3. Aproximar el error de *test* promediando sobre los ECM: $CV_{(k)} = \frac{1}{k}\sum_{i=1}^{k}EMC_{i}$

Algunas observaciones relevantes:

1. Si hacemos $k = N$ entonces el método se llama *Leave-One-Out Cross-Validation* (LOOCV) (pero esto es costoso computancionalmente)
2. En general se usan valores de $k=5$ o $k = 10$. ¿Por qué?
3. Todos estos métodos asumen que las observaciones son i.i.d...


### Overfitting, modelos e hiperparametros

Con las estrategias de holdout tenemos una forma de aproximar el error de testeo y detectar casos de overfitting: ¿cómo reducimos este problema?

Podemos probar distintos modelos (*no free-lunch theorem*)

Podemos optimizar aquellos parámetros de un mismo modelo que no se derivan del proceso de aprendizaje: los *hiperparámetros*









In [ ]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=5, random_state=0)
df_mse_cv = pd.DataFrame()
for max_depth in max_depths:
    df_row = {'max_depth': max_depth}
    max_depth_mses = []
    for train_index, test_index in kf.split(X, y):
        m = DecisionTreeRegressor(max_depth=max_depth)
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y[train_index], y[test_index]
        clf = m.fit(X_train, y_train)
        y_hat = clf.predict(X_test)
        mse = score(y_test, y_hat)
        max_depth_mses.append(mse)
    df_row['max_depth_mse_cv'] = np.mean(max_depth_mses)
    df_mse_cv = df_mse_cv.append(df_row, ignore_index=True)

In [ ]:
df_mse_cv

In [ ]:
df_mse_cv[df_mse_cv['max_depth_mse_cv'] == df_mse_cv['max_depth_mse_cv'].min()]

### Ejercicios Selección de Modelos

1. **Overfitting y validation set**
  1. Escriba una función que separe el conjunto de datos en entrenamiento y validación a partir de un parámetro que determina el tamaño del segundo conjunto.
  2. Utilice ahora la función `model_selection.train_test_split` para realizar la separación del inciso anterior.
  3. Empleando alguna de las dos funciones de los incisos anteriores investigue la existencia de overfitting para distintos tamaños del conjunto de validación.
2. **Más feature engineering: binning y dummies**
  1. Cree una nueva variable `age` que refleje la antigüedad de las casas al ser vendidas.
  2. Genere nuevos features a partir de esta columna usando una estrategia de `binning` (y `dummies`).
  3. Fittee un nuevo modelo Lasso con estos features ampliados y compute el ECM. Comparelo con el modelo completo sin feature engineering.
    1. ¿Estas transformaciones las debe hacer también en el conjunto de validacion?
3. **Validación cruzada e hiperparametros**
  1. Utilizando ahora la función `cross_val_score` del módulo `sklearn.model_section` repita el ejercicio de validación cruzada y obtenga los ECM para cada valor del hiperparámetros $alpha$ (en logaritmos).
  2. Grafique los errores cuadráticos medios resultantes.
  3. A partir del modelo `LassoCV` obtenga nuevamente el hiperparámetro óptimo.




### Ejercicio 1: Overfitting  y validation set

In [ ]:
np.random.rand(10)

In [ ]:
msk = np.random.rand(len(X)) <= 0.2

In [ ]:
msk

In [ ]:
msk.sum()

In [ ]:
X[~msk]

In [ ]:
def train_test_split(X, y, test_size=0.2):
    bool_mask = np.random.rand(len(X)) <= (1 - test_size)
    X_train, X_test = X[bool_mask], X[~bool_mask]
    y_train, y_test = y[bool_mask], y[~bool_mask]
    return X_train, X_test, y_train, y_test

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
from sklearn import model_selection

In [ ]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.1)

In [ ]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [ ]:
for i in np.arange(0.1, 0.6, 0.1):
    print(f"Test size {i}")
    X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=i)
    print(f"Model lineal")
    fit_pipeline(X_train, y_train, X_test, y_test, 'lr')
    print(f"Model Decision Tree")
    fit_pipeline(X_train, y_train, X_test, y_test, 'dt')


### Ejercicio 2: binning y dummies

In [ ]:
X_bd = X.copy()

In [ ]:
X_bd

In [ ]:
# TODO: implementar X_bd['age'] = 
X_bd

In [ ]:
X_bd['age'].value_counts()

In [ ]:
bins = [-2, 0, 5, 10, 25, 50, 75, 100, 100000]
labels = ['<1', '1-5', '6-10', '11-25', '26-50', '51-75', '76-100', '>100']
#TODO: implementar X_bd['age_binned'] = 
X_bd[['age', 'age_binned']]

In [ ]:
# TODO implementar dummies X_bd = 
X_bd

In [ ]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(X_bd, y, test_size=0.2)

In [ ]:
fit_pipeline(X_train, y_train, X_test, y_test, 'dt')

In [ ]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.2)

In [ ]:
fit_pipeline(X_train, y_train, X_test, y_test, 'dt')

### Ejercicio 3: Validacion cruzada e hiperparametros

In [ ]:
from sklearn.model_selection import cross_val_score

In [ ]:
df_mse_sk = pd.DataFrame()
for max_depth in max_depths:
    df_mse_sk_row = {'max_depth': max_depth}
    cv_score = -cross_val_score(DecisionTreeRegressor(max_depth=max_depth, random_state=0), X, y, cv=5, 
                               scoring='neg_mean_squared_error')
    df_mse_sk_row['cv'] = np.mean(cv_score)
    df_mse_sk = df_mse_sk.append(df_mse_sk_row, ignore_index=True)

In [ ]:
df_mse_sk

In [ ]:
df_mse_sk[df_mse_sk['cv'] == df_mse_sk['cv'].min()]

## Modelos de Ensamble y Random Forest

### Sobre train, validation, test split

¿Podemos overfittear el conjunto de validación?

Si realizamos muchas pruebas posiblemente si

Por eso en la práctica trabajamos con 3 conjuntos: entrenamiento, validación y testeo.

![](https://raw.githubusercontent.com/petobens/introduccion-ml-aplicado/master/figures/ml-claro-1/tvt.png)

En general se reserva 20% para testing y el remanente se divide en 80% entrenamiento y 20% validación.

Con muchos datos los porcentajes son menores. Es una cuestión de representatividad y no de números.

![](https://raw.githubusercontent.com/petobens/introduccion-ml-aplicado/master/figures/ml-claro-1/sample_size.png)


### Tradeoff de Sesgo-Varianza

¿Cuál es el valor esperado de ECM en una observación de testeo?

Es decir cuál es el ECM promedio si uno estima sucesivas veces $f$ usando muchos conjuntos de entrenamiento y evaluando en cada $x_{0}$ del conjunto de test:

$$E(y_{0} - \hat{f}(x_{0}))^{2} = Var(\hat{f}(x_{0})) +
        \left[Sesgo(\hat{f}(x_{0}))\right]^{2} + Var(\varepsilon)$$

Donde:

La varianza remite a cuánto cambiaría $\hat{f}$ si estimaramos con otro conjunto de entrenamiento (idealmente queremos que sea poco).

El sesgo alude a si $\hat{f}$ está errando consistentemente en las predicciones ($Sesgo(\hat{f}(x_{0})) = E[\hat{f}(x_{0})] - y_{0}$).

El ideal: tener un sesgo bajo y una varianza baja. ¿Es esto posible?

![](https://prateekvjoshi.files.wordpress.com/2015/10/3-bulls-eye.png)

![](https://miro.medium.com/max/492/1*blqnaVEu6Hbc-5ZYeDnU9Q.png)

¿Por qué hay un tradeoff?

- Es fácil obtener un método con bajo sesgo y alta varianza (dibujando una curva que pase por todos los puntos)
- Es fácil obtener un método con bajo o nula varianza y alto sesgo (fitteando una constante)

En general vale lo siguiente:

- Métodos más complejos tienen alta varianza y bajo sesgo.
- El fenómeno de *overfitting* se asocia a escenarios justamente de alta varianza y bajo sesgo.










### Bagging y Árboles aleatorios

**Bagging**

En general vale que agregar observaciones reduce la varianza.

Podemos crear $B$ conjuntos de entrenamiento muestreando de forma aleatoria (y con reposición) del conjunto entrenamiento original (*bootstraping*).

Si luego entrenamos sobre cada conjunto y promediamos reducimos la varianza (y tenemos bajo sesgo)
$$\hat{f}_{bag}(x) = \frac{1}{B}\sum_{b=1}^{B}\hat{f}^{*b}(x)$$

La predicción final es la clase más votada por los $B$ árboles (regla mayoritaria).

**Random Forests**

Ideado por Breiman con el objeto de mejorar aun más la performance predictiva.

Al igual que en bagging se crean $B$ conjuntos de entrenamiento muestreando de forma aleatoria (y con reposición) del conjunto entrenamiento original.

Para cada conjunto se hacer particionamiento recursivo pero al elegir los cortes se emplea un subconjunto aleatorio de la totalidad de atributos (*descorrelacionar*)

In [ ]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.2)

In [ ]:
fit_pipeline(X_train, y_train, X_test, y_test, 'rf')


# Ejercicio

Utilizando lo aprendido en este notebook y el anterior. 
Desarolle un pipeline para predecir el precio de casas del *Ames Housing dataset*.

El link a la competencia es el siguiente: 
https://www.kaggle.com/c/home-data-for-ml-course/data

Para obetener el dataset de train desde el notebook puede utilizar los siguientes comandos para dejar el archivo `train.csv` en el directorio actual.

```
!kaggle datasets download -d dansbecker/home-data-for-ml-course
!unzip home-data-for-ml-course.zip
```

Comience por explorar los datos y fitear un modelo simple y calcular la métrica de error definida para la competencia para tener un baseline.

Luego, vaya realizando feature engineering progresivamente y verificando como varía su métrica de performance. 

Eventualmente, cuando esté satisfecho con la performance, bajer el archivo `test.csv` de la web de la competencia y verifique la capacidad de generalización de su modelo.

Como paso final, puede submitear su respuesta a la competencia siguiendo el formato descripto en la misma https://www.kaggle.com/c/home-data-for-ml-course/overview/evaluation

Para esto hay que generar un archivo CSV con el formato designado y subirlo manualmente a https://www.kaggle.com/c/home-data-for-ml-course/submit

O también puede probar utilizando la commandline app de kaggle de la siguiente manera:


```
!kaggle competitions submit home-data-for-ml-course -f my_submission.csv -m "First Submission @pferrari"
```


Referencia útil:

https://stackoverflow.com/questions/49394737/exporting-data-from-google-colab-to-local-machine/49397357